In [2]:
! pip install transformers
import pandas as pd
import numpy as np
import os
path = "/content/"
offer_details = pd.read_excel(path+"offer_retailer.xlsx")
brand_cat = pd.read_excel(path+"brand_category.xlsx")
cat_details = pd.read_excel(path+"categories.xlsx")

In [3]:
merge_1 = pd.merge(offer_details, brand_cat, on='BRAND')

In [4]:
merged_df = pd.merge(merge_1, cat_details,  left_on='BRAND_BELONGS_TO_CATEGORY', right_on='PRODUCT_CATEGORY')

In [5]:
merged_df = merged_df.drop(columns = ["RECEIPTS", "CATEGORY_ID"])

In [6]:
from transformers import BertTokenizer, BertModel
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = BertModel.from_pretrained('bert-base-uncased')
import torch

# Set device to CUDA if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Move the model to the GPU
bert_model.to(device)

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
  

In [7]:
def bert_embedding_gen(input):
  bert_embeddings = []
  for sentence in input:
  # Tokenize and move tokens to the GPU
    tokens = bert_tokenizer(sentence, return_tensors='pt').to(device)

    # Forward pass on the GPU
    with torch.no_grad():
        outputs = bert_model(**tokens)

    # Move embeddings to CPU and append to the list
    bert_embeddings.append(outputs.last_hidden_state.mean(dim=1).cpu().numpy())
  return bert_embeddings

In [8]:
merged_df.fillna('', inplace=True)
merged_df['RETAILER_bert'] = bert_embedding_gen(merged_df['RETAILER'])
merged_df['BRAND_bert'] = bert_embedding_gen(merged_df['BRAND'])
merged_df['BRAND_cat'] = bert_embedding_gen(merged_df['PRODUCT_CATEGORY'])
merged_df['BRAND_par_cat'] = bert_embedding_gen(merged_df['IS_CHILD_CATEGORY_TO'])

In [9]:
def cosine_simm(x, y):
  from sklearn.metrics.pairwise import cosine_similarity

  # Assuming 'embedding_col1' and 'embedding_col2' are columns with BERT embeddings
  embeddings1 = x.tolist()
  embeddings2 = y.tolist()

  # Reshape the embeddings to 1D arrays
  embeddings1 = [embedding.reshape(1, -1) for embedding in embeddings1]
  embeddings2 = [embedding.reshape(1, -1) for embedding in embeddings2]

  # Calculate cosine similarity for each pair
  similarities = [cosine_similarity(e1, e2)[0][0] for e1, e2 in zip(embeddings1, embeddings2)]
  return similarities

In [10]:
text = str(input("Enter Your offer search input"))
tokens = bert_tokenizer(text, return_tensors='pt').to(device)
with torch.no_grad():
 outputs = bert_model(**tokens)
input_embedding = outputs.last_hidden_state.mean(dim=1).cpu().numpy()
merged_df["input_emb"] = [input_embedding] * merged_df.shape[0]
merged_df['combined_bert'] = merged_df['RETAILER_bert'] + merged_df['BRAND_bert'] + merged_df['BRAND_cat'] + merged_df['BRAND_par_cat']
merged_df['score_RETAILER_bert']= cosine_simm(merged_df["input_emb"], merged_df['RETAILER_bert'])
merged_df['score_BRAND_bert'] = cosine_simm(merged_df["input_emb"], merged_df['BRAND_bert'])
merged_df['score_BRAND_cat'] = cosine_simm(merged_df["input_emb"], merged_df['BRAND_cat'])
merged_df['score_BRAND_par_cat'] = cosine_simm(merged_df["input_emb"], merged_df['BRAND_par_cat'])
merged_df['score_combined_bert'] = cosine_simm(merged_df["input_emb"], merged_df['combined_bert'])
logits = merged_df[['score_RETAILER_bert', 'score_BRAND_bert', 'score_BRAND_cat', 'score_BRAND_par_cat', 'score_combined_bert']].max(axis=1)
merged_df['OFFER'][np.argmax(logits)]

Enter Your offer search inputRalphs cookies


'Butterball, select varieties, spend $10 at Ralphs'